# Предобработка документов вакансий

In [1]:
import json
import requests

from bs4 import BeautifulSoup
from elasticsearch import Elasticsearch
from pathlib import Path

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
WORK_DIR = Path("/home/ephobia/Desktop/HH")
FILE_PATH = WORK_DIR / "data"
INDEX_NAME = "vacancies"

ES = Elasticsearch(hosts="http://localhost:9200")

## Загрузка данных на ElasticSearch

Данные предатавляют собой структурированные документы с вложенными полями, что затрудняет их обработку в реляционных базах данных. Также объем данных слишком большой, чтобы проводить последующий EDA без возможности поиска по полям документов.

В связи с этим для хранения документов мною была выбрана нереляционная СУБД Elasticsearch. Перенесем базу данных в подготовленный индекс.

In [3]:
for file in FILE_PATH.rglob("*"):
    text = file.read_text()
    json_text = json.loads(text)
    ES.index(index=INDEX_NAME, id=file.stem, document=json_text)

In [4]:
response = ES.search(index=INDEX_NAME, query={
    "match_all": {}
}, size = 30000)

documents = response["hits"]["hits"]

In [5]:
len(documents)

29522

## Первичная обработка данных

Некоторые из полей требуют предварительной обработки для последующего анализа. Дополним структуру документа новыми полями с обработанными данными.

### Зарплатная вилка

Формат представления зарплаты включает в себя следующие особенности:

1. Зарплата может быть указана как диапазон значений или не быть указана совсем.
2. Зарплата может быть указана в различных валютах.
3. Встречаются случаи указания зарплаты как до, так и после вычета налогов (это также отображено в структуре документа вакансии)

Задача: привести зарплаты в единый диапазон значений (в рублях после вычета налогов) и попробовать сформулировать вилку одним значением (в данном случае - максимальное значение в вилке).

API HeadHunter включает в себя эндпоинт для получения коэффициента отношения данной валюты к курсы рубля. С помощью него довольно просто получить финальное значение в рублях.

Новые поля:
* salary/to_final
* salary/from_final
* max_final_salary
* max_salary

In [6]:
def get_currencies():
    currencies = {}
    dictionaries = requests.get('https://api.hh.ru/dictionaries').json()
    for currency in dictionaries['currency']:
        currencies[currency['code']] = currency['rate']
    return currencies

In [7]:
CURRENCIES = get_currencies()

In [8]:
def get_rub_final_salary(document, salary):
    salary /= CURRENCIES[document["_source"]['salary']['currency']]
    if document["_source"]['salary']['gross']:
        salary -= salary * 0.13
    return salary

### Текст вакансии

В структуре документа представлено два поля с описанием вакансии: обычное описание и брендированное. Они могут быть заполненны одновременно, но разным текстом.

Данные поля содержат текст в html разметке. Задача: очистить текст от разметки для последующего текстуального анализа.

Новые поля:

* postpoc_desc
* postpoc_branded_desc

In [9]:
def get_text_from_html(html_doc):
    if html_doc is not None:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return soup.get_text()
    return None

In [10]:
for document in documents:
    document["_source"]["postpoc_desc"] = get_text_from_html(document["_source"]["description"])
    document["_source"]["postpoc_branded_desc"] = get_text_from_html(document["_source"]["branded_description"])
    
    max_salary = -1
    if document["_source"]['salary']:
        
        if document["_source"]['salary']['to']:
            to_final_salary = get_rub_final_salary(document, document["_source"]['salary']['to'])
            if document["_source"]['salary']['to'] > max_salary:
                max_salary = document["_source"]['salary']['to']
            document["_source"]['salary']['to_final'] = to_final_salary

        if document["_source"]['salary']['from']:
            from_final_salary = get_rub_final_salary(document, document["_source"]['salary']['from'])
            if from_final_salary > document["_source"]['salary']['from']:
                max_salary = document["_source"]['salary']['from']
            document["_source"]['salary']['from_final'] = to_final_salary
            
        if max_salary > 0: 
            document["_source"]['max_final_salary'] = get_rub_final_salary(document, max_salary)
        else:
            max_salary = None
            document["_source"]['max_final_salary'] = max_salary

    document["_source"]['max_salary'] = max_salary 

## Обновление  документов в базе данных

In [11]:
def get_doc_to_update(document):
    source = document["_source"]
    return {"doc": source}

In [12]:
for document in documents:
    response = ES.update(
        index=INDEX_NAME,
        id=document["_id"],
        body=get_doc_to_update(document)
    )

/home/ephobia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  """
